# Projet de Web Scraping - Dakar Auto

Ce notebook va scraper et nettoyer les données de trois catégories sur dakar-auto.com:
1. Voitures
2. Motos et Scooters
3. Location de voitures

In [ ]:
# Installation des bibliothèques nécessaires
# Exécutez cette cellule une seule fois
import sys
!{sys.executable} -m pip install requests beautifulsoup4 pandas lxml

In [ ]:
# Import des bibliothèques
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from typing import List, Dict

## Fonctions utilitaires pour le scraping

In [ ]:
def get_page_content(url: str) -> BeautifulSoup:
    """Récupère le contenu HTML d'une page"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'lxml')
    except Exception as e:
        print(f"Erreur lors de la récupération de {url}: {e}")
        return None

def clean_text(text: str) -> str:
    """Nettoie le texte en supprimant les espaces superflus"""
    if text:
        return re.sub(r'\s+', ' ', text.strip())
    return ""

def extract_number(text: str) -> str:
    """Extrait les nombres d'un texte"""
    if text:
        numbers = re.findall(r'\d+', text.replace(' ', ''))
        return ''.join(numbers) if numbers else ""
    return ""

## 1. Scraping des Voitures

In [ ]:
def scrape_voitures(base_url: str, max_pages: int = 2773) -> List[Dict]:
    """
    Scrape les données des voitures
    Variables: marque, année, prix, adresse, kilométrage, boite vitesse, carburant, propriétaire
    """
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"Scraping page {page}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url
        soup = get_page_content(url)
        
        if not soup:
            break
            
        # Trouver tous les articles/annonces avec la vraie structure
        articles = soup.find_all('div', class_='listings-cards__list-item')
        
        if not articles:
            print(f"Aucun article trouvé sur la page {page}")
            break
            
        for article in articles:
            try:
                data = {}
                
                # Marque (titre de l'annonce)
                title_elem = article.find('h2', class_='listing-card__header__title')
                if title_elem:
                    title_link = title_elem.find('a')
                    data['marque'] = clean_text(title_link.get_text()) if title_link else clean_text(title_elem.get_text())
                else:
                    data['marque'] = ""
                
                # Prix
                price_elem = article.find('h3', class_='listing-card__header__price')
                data['prix'] = clean_text(price_elem.get_text()) if price_elem else ""
                
                # Adresse (ville + province)
                address_parts = []
                town_elem = article.find('span', class_='town-suburb')
                if town_elem:
                    address_parts.append(clean_text(town_elem.get_text()))
                province_elem = article.find('span', class_='province')
                if province_elem:
                    address_parts.append(clean_text(province_elem.get_text()))
                data['adresse'] = ' '.join(address_parts)
                
                # Initialiser les caractéristiques
                data['année'] = ""
                data['kilométrage'] = ""
                data['boite_vitesse'] = ""
                data['carburant'] = ""
                
                # Extraire l'année du titre si présent
                if data['marque']:
                    year_match = re.search(r'\b(19|20)\d{2}\b', data['marque'])
                    if year_match:
                        data['année'] = year_match.group()
                
                # Caractéristiques dans les li de la liste
                specs = article.find_all('li', class_='listing-card__attribute')
                for spec in specs:
                    spec_text = clean_text(spec.get_text()).lower()
                    
                    # Kilométrage (contient "km")
                    if 'km' in spec_text and 'km' not in data['kilométrage']:
                        data['kilométrage'] = clean_text(spec.get_text())
                    
                    # Boîte de vitesse
                    if 'automatique' in spec_text or 'manuelle' in spec_text:
                        data['boite_vitesse'] = clean_text(spec.get_text())
                    
                    # Carburant
                    if any(word in spec_text for word in ['essence', 'diesel', 'hybride', 'électrique']):
                        data['carburant'] = clean_text(spec.get_text())
                
                # Propriétaire (dans le texte "Par [nom]")
                author_elem = article.find('p', class_='time-author')
                if author_elem:
                    author_link = author_elem.find('a')
                    if author_link:
                        author_text = clean_text(author_link.get_text())
                        # Enlever "Par " du début
                        data['propriétaire'] = author_text.replace('Par ', '').strip()
                    else:
                        data['propriétaire'] = clean_text(author_elem.get_text()).replace('Par ', '').strip()
                else:
                    data['propriétaire'] = ""
                
                all_data.append(data)
                
            except Exception as e:
                print(f"Erreur lors du traitement d'un article: {e}")
                continue
        
        time.sleep(1)  # Pause pour ne pas surcharger le serveur
    
    print(f"Total voitures scrapées: {len(all_data)}")
    return all_data

## 2. Scraping des Motos et Scooters

In [ ]:
def scrape_motos(base_url: str, max_pages: int = 55) -> List[Dict]:
    """
    Scrape les données des motos et scooters
    Variables: marque, année, prix, adresse, kilométrage, propriétaire
    """
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"Scraping page {page}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url
        soup = get_page_content(url)
        
        if not soup:
            break
            
        # Trouver tous les articles/annonces
        articles = soup.find_all('div', class_='listings-cards__list-item')
        
        if not articles:
            print(f"Aucun article trouvé sur la page {page}")
            break
            
        for article in articles:
            try:
                data = {}
                
                # Marque (titre de l'annonce)
                title_elem = article.find('h2', class_='listing-card__header__title')
                if title_elem:
                    title_link = title_elem.find('a')
                    data['marque'] = clean_text(title_link.get_text()) if title_link else clean_text(title_elem.get_text())
                else:
                    data['marque'] = ""
                
                # Prix
                price_elem = article.find('h3', class_='listing-card__header__price')
                data['prix'] = clean_text(price_elem.get_text()) if price_elem else ""
                
                # Adresse (ville + province)
                address_parts = []
                town_elem = article.find('span', class_='town-suburb')
                if town_elem:
                    address_parts.append(clean_text(town_elem.get_text()))
                province_elem = article.find('span', class_='province')
                if province_elem:
                    address_parts.append(clean_text(province_elem.get_text()))
                data['adresse'] = ' '.join(address_parts)
                
                # Initialiser les caractéristiques
                data['année'] = ""
                data['kilométrage'] = ""
                
                # Extraire l'année du titre si présent
                if data['marque']:
                    year_match = re.search(r'\b(19|20)\d{2}\b', data['marque'])
                    if year_match:
                        data['année'] = year_match.group()
                
                # Caractéristiques dans les li
                specs = article.find_all('li', class_='listing-card__attribute')
                for spec in specs:
                    spec_text = clean_text(spec.get_text()).lower()
                    
                    # Kilométrage
                    if 'km' in spec_text and 'km' not in data['kilométrage']:
                        data['kilométrage'] = clean_text(spec.get_text())
                
                # Propriétaire
                author_elem = article.find('p', class_='time-author')
                if author_elem:
                    author_link = author_elem.find('a')
                    if author_link:
                        author_text = clean_text(author_link.get_text())
                        data['propriétaire'] = author_text.replace('Par ', '').strip()
                    else:
                        data['propriétaire'] = clean_text(author_elem.get_text()).replace('Par ', '').strip()
                else:
                    data['propriétaire'] = ""
                
                all_data.append(data)
                
            except Exception as e:
                print(f"Erreur lors du traitement d'un article: {e}")
                continue
        
        time.sleep(1)
    
    print(f"Total motos scrapées: {len(all_data)}")
    return all_data

## 3. Scraping des Locations de Voitures

In [ ]:
def scrape_locations(base_url: str, max_pages: int = 9) -> List[Dict]:
    """
    Scrape les données des locations de voitures
    Variables: marque, année, prix, adresse, propriétaire
    """
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"Scraping page {page}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url
        soup = get_page_content(url)
        
        if not soup:
            break
            
        # Trouver tous les articles/annonces
        articles = soup.find_all('div', class_='listings-cards__list-item')
        
        if not articles:
            print(f"Aucun article trouvé sur la page {page}")
            break
            
        for article in articles:
            try:
                data = {}
                
                # Marque (titre de l'annonce)
                title_elem = article.find('h2', class_='listing-card__header__title')
                if title_elem:
                    title_link = title_elem.find('a')
                    data['marque'] = clean_text(title_link.get_text()) if title_link else clean_text(title_elem.get_text())
                else:
                    data['marque'] = ""
                
                # Prix
                price_elem = article.find('h3', class_='listing-card__header__price')
                data['prix'] = clean_text(price_elem.get_text()) if price_elem else ""
                
                # Adresse (ville + province)
                address_parts = []
                town_elem = article.find('span', class_='town-suburb')
                if town_elem:
                    address_parts.append(clean_text(town_elem.get_text()))
                province_elem = article.find('span', class_='province')
                if province_elem:
                    address_parts.append(clean_text(province_elem.get_text()))
                data['adresse'] = ' '.join(address_parts)
                
                # Année - extraire du titre
                data['année'] = ""
                if data['marque']:
                    year_match = re.search(r'\b(19|20)\d{2}\b', data['marque'])
                    if year_match:
                        data['année'] = year_match.group()
                
                # Propriétaire
                author_elem = article.find('p', class_='time-author')
                if author_elem:
                    author_link = author_elem.find('a')
                    if author_link:
                        author_text = clean_text(author_link.get_text())
                        data['propriétaire'] = author_text.replace('Par ', '').strip()
                    else:
                        data['propriétaire'] = clean_text(author_elem.get_text()).replace('Par ', '').strip()
                else:
                    data['propriétaire'] = ""
                
                all_data.append(data)
                
            except Exception as e:
                print(f"Erreur lors du traitement d'un article: {e}")
                continue
        
        time.sleep(1)
    
    print(f"Total locations scrapées: {len(all_data)}")
    return all_data

## 4. Fonction de nettoyage des données

In [ ]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Nettoie et standardise un DataFrame"""
    
    # Supprimer les lignes complètement vides
    df = df.dropna(how='all')
    
    # Nettoyer les prix
    if 'prix' in df.columns:
        df['prix'] = df['prix'].apply(lambda x: extract_number(str(x)) if pd.notna(x) else "")
        # Convertir en numérique si possible
        df['prix_numeric'] = pd.to_numeric(df['prix'], errors='coerce')
    
    # Nettoyer les kilométrages
    if 'kilométrage' in df.columns:
        df['kilométrage_clean'] = df['kilométrage'].apply(lambda x: extract_number(str(x)) if pd.notna(x) else "")
        df['kilométrage_numeric'] = pd.to_numeric(df['kilométrage_clean'], errors='coerce')
    
    # Nettoyer les années
    if 'année' in df.columns:
        df['année'] = df['année'].apply(lambda x: extract_number(str(x)) if pd.notna(x) else "")
        df['année_numeric'] = pd.to_numeric(df['année'], errors='coerce')
    
    # Supprimer les doublons basés sur marque et prix
    if 'marque' in df.columns and 'prix' in df.columns:
        df = df.drop_duplicates(subset=['marque', 'prix'], keep='first')
    
    # Réinitialiser l'index
    df = df.reset_index(drop=True)
    
    return df

## 5. Exécution du scraping

In [ ]:
# URLs à scraper
url_voitures = "https://dakar-auto.com/senegal/voitures-4"
url_motos = "https://dakar-auto.com/senegal/motos-and-scooters-3"
url_locations = "https://dakar-auto.com/senegal/location-de-voitures-19"

# Nombre de pages à scraper (ajustez selon vos besoins)
MAX_PAGES = 3

print("="*60)
print("SCRAPING DES VOITURES")
print("="*60)
voitures_data = scrape_voitures(url_voitures)
df_voitures = pd.DataFrame(voitures_data)
df_voitures = clean_dataframe(df_voitures)

print("\n" + "="*60)
print("SCRAPING DES MOTOS ET SCOOTERS")
print("="*60)
motos_data = scrape_motos(url_motos)
df_motos = pd.DataFrame(motos_data)
df_motos = clean_dataframe(df_motos)

print("\n" + "="*60)
print("SCRAPING DES LOCATIONS")
print("="*60)
locations_data = scrape_locations(url_locations)
df_locations = pd.DataFrame(locations_data)
df_locations = clean_dataframe(df_locations)

print("\n" + "="*60)
print("RÉSUMÉ")
print("="*60)
print(f"Voitures scrapées: {len(df_voitures)}")
print(f"Motos scrapées: {len(df_motos)}")
print(f"Locations scrapées: {len(df_locations)}")

SCRAPING DES VOITURES
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scrapin

## 6. Affichage et analyse des données

In [ ]:
# Afficher les premières lignes de chaque dataset
print("="*60)
print("VOITURES - Premières lignes")
print("="*60)
print(df_voitures.head())
print(f"\nShape: {df_voitures.shape}")
print(f"Colonnes: {list(df_voitures.columns)}")

print("\n" + "="*60)
print("MOTOS - Premières lignes")
print("="*60)
print(df_motos.head())
print(f"\nShape: {df_motos.shape}")
print(f"Colonnes: {list(df_motos.columns)}")

print("\n" + "="*60)
print("LOCATIONS - Premières lignes")
print("="*60)
print(df_locations.head())
print(f"\nShape: {df_locations.shape}")
print(f"Colonnes: {list(df_locations.columns)}")

VOITURES - Premières lignes
                     marque      prix                    adresse année  \
0          Lexus GX460 2020  38000000         Ouest Foire, Dakar  2020   
1     Mercedes GLC-300 2018  15000000         Ouest Foire, Dakar  2018   
2               Kia K5 2018   5500000                Mbao, Dakar  2018   
3  Maserati Levante-Q4 2019  21000000   Sicap Sacré Coeur, Dakar  2019   
4     Hyundai Santa Fe 2017   9000000  Mermoz Sacré-Coeur, Dakar  2017   

  kilométrage boite_vitesse carburant                propriétaire  \
0    55493 km   Automatique   Essence                Ulrich MIÉRÉ   
1    66485 km   Automatique   Essence                Ulrich MIÉRÉ   
2    89220 km   Automatique   Essence  cheikh ahmed tidiane toure   
3    11000 km   Automatique    Diesel              Mansour Diagne   
4    92000 km   Automatique   Essence                    Mme Diop   

   prix_numeric kilométrage_clean  kilométrage_numeric  année_numeric  
0    38000000.0             55493       

---

# PARTIE 2: Scraping SANS nettoyage de TOUTES les pages

Cette section scrappe toutes les pages disponibles et conserve les données brutes sans nettoyage.

## Fonction de détection automatique du nombre de pages

In [ ]:
def get_total_pages(base_url: str) -> int:
    """Détecte automatiquement le nombre total de pages"""
    soup = get_page_content(base_url)
    if not soup:
        return 1
    
    try:
        # Chercher le lien "dernière page"
        pagination = soup.find('nav', class_='paginator')
        if pagination:
            last_page_link = pagination.find('a', string=lambda s: s and 'dernière page' in s.lower())
            if last_page_link and last_page_link.get('href'):
                # Extraire le numéro de page de l'URL
                match = re.search(r'page=(\d+)', last_page_link['href'])
                if match:
                    return int(match.group(1))
            
            # Si pas de lien "dernière page", chercher le plus grand numéro de page
            page_links = pagination.find_all('a', class_='page-link')
            max_page = 1
            for link in page_links:
                text = link.get_text().strip()
                if text.isdigit():
                    max_page = max(max_page, int(text))
            return max_page
    except Exception as e:
        print(f"Erreur lors de la détection du nombre de pages: {e}")
    
    return 1

## Fonctions de scraping SANS nettoyage - Toutes les pages

In [ ]:
def scrape_voitures_brut(base_url: str, max_pages: int = None) -> List[Dict]:
    """
    Scrape les données brutes des voitures (SANS NETTOYAGE) - TOUTES LES PAGES
    Variables: marque, année, prix, adresse, kilométrage, boite vitesse, carburant, propriétaire
    """
    # Détecter automatiquement le nombre de pages si non spécifié
    if max_pages is None:
        print("🔍 Détection du nombre total de pages...")
        max_pages = get_total_pages(base_url)
        print(f"✓ {max_pages} pages détectées\n")
    
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"📄 Scraping page {page}/{max_pages}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url
        soup = get_page_content(url)
        
        if not soup:
            print(f"❌ Impossible de récupérer la page {page}, arrêt.")
            break
            
        # Trouver tous les articles
        articles = soup.find_all('div', class_='listings-cards__list-item')
        
        if not articles:
            print(f"⚠️ Aucun article trouvé sur la page {page}, arrêt.")
            break
            
        for article in articles:
            try:
                data = {}
                
                # V1: Marque - BRUT
                title_elem = article.find('h2', class_='listing-card__header__title')
                if title_elem:
                    title_link = title_elem.find('a')
                    data['marque'] = title_link.get_text().strip() if title_link else title_elem.get_text().strip()
                else:
                    data['marque'] = ""
                
                # V2: Année
                data['année'] = ""
                if data['marque']:
                    year_match = re.search(r'\b(19|20)\d{2}\b', data['marque'])
                    if year_match:
                        data['année'] = year_match.group()
                
                # V3: Prix - BRUT
                price_elem = article.find('h3', class_='listing-card__header__price')
                data['prix'] = price_elem.get_text().strip() if price_elem else ""
                
                # V4: Adresse - BRUT
                address_parts = []
                town_elem = article.find('span', class_='town-suburb')
                if town_elem:
                    address_parts.append(town_elem.get_text().strip())
                province_elem = article.find('span', class_='province')
                if province_elem:
                    address_parts.append(province_elem.get_text().strip())
                data['adresse'] = ' '.join(address_parts)
                
                # V5, V6, V7: Caractéristiques - BRUT
                data['kilométrage'] = ""
                data['boite_vitesse'] = ""
                data['carburant'] = ""
                
                specs = article.find_all('li', class_='listing-card__attribute')
                for spec in specs:
                    spec_text = spec.get_text().strip()
                    spec_lower = spec_text.lower()
                    
                    if 'km' in spec_lower and not data['kilométrage']:
                        data['kilométrage'] = spec_text
                    
                    if 'automatique' in spec_lower or 'manuelle' in spec_lower:
                        data['boite_vitesse'] = spec_text
                    
                    if any(word in spec_lower for word in ['essence', 'diesel', 'hybride', 'électrique']):
                        data['carburant'] = spec_text
                
                # V8: Propriétaire - BRUT
                author_elem = article.find('p', class_='time-author')
                if author_elem:
                    author_link = author_elem.find('a')
                    data['propriétaire'] = author_link.get_text().strip() if author_link else author_elem.get_text().strip()
                else:
                    data['propriétaire'] = ""
                
                all_data.append(data)
                
            except Exception as e:
                print(f"⚠️ Erreur article: {e}")
                continue
        
        if page < max_pages:
            time.sleep(1)
    
    print(f"\n✅ Total voitures scrapées: {len(all_data)}")
    return all_data

In [ ]:
def scrape_motos_brut(base_url: str, max_pages: int = None) -> List[Dict]:
    """
    Scrape les données brutes des motos (SANS NETTOYAGE) - TOUTES LES PAGES
    Variables: marque, année, prix, adresse, kilométrage, propriétaire
    """
    if max_pages is None:
        print("🔍 Détection du nombre total de pages...")
        max_pages = get_total_pages(base_url)
        print(f"✓ {max_pages} pages détectées\n")
    
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"📄 Scraping page {page}/{max_pages}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url
        soup = get_page_content(url)
        
        if not soup:
            print(f"❌ Impossible de récupérer la page {page}, arrêt.")
            break
            
        articles = soup.find_all('div', class_='listings-cards__list-item')
        
        if not articles:
            print(f"⚠️ Aucun article trouvé sur la page {page}, arrêt.")
            break
            
        for article in articles:
            try:
                data = {}
                
                # V1: Marque - BRUT
                title_elem = article.find('h2', class_='listing-card__header__title')
                if title_elem:
                    title_link = title_elem.find('a')
                    data['marque'] = title_link.get_text().strip() if title_link else title_elem.get_text().strip()
                else:
                    data['marque'] = ""
                
                # V2: Année
                data['année'] = ""
                if data['marque']:
                    year_match = re.search(r'\b(19|20)\d{2}\b', data['marque'])
                    if year_match:
                        data['année'] = year_match.group()
                
                # V3: Prix - BRUT
                price_elem = article.find('h3', class_='listing-card__header__price')
                data['prix'] = price_elem.get_text().strip() if price_elem else ""
                
                # V4: Adresse - BRUT
                address_parts = []
                town_elem = article.find('span', class_='town-suburb')
                if town_elem:
                    address_parts.append(town_elem.get_text().strip())
                province_elem = article.find('span', class_='province')
                if province_elem:
                    address_parts.append(province_elem.get_text().strip())
                data['adresse'] = ' '.join(address_parts)
                
                # V5: Kilométrage - BRUT
                data['kilométrage'] = ""
                specs = article.find_all('li', class_='listing-card__attribute')
                for spec in specs:
                    spec_text = spec.get_text().strip()
                    if 'km' in spec_text.lower() and not data['kilométrage']:
                        data['kilométrage'] = spec_text
                        break
                
                # V6: Propriétaire - BRUT
                author_elem = article.find('p', class_='time-author')
                if author_elem:
                    author_link = author_elem.find('a')
                    data['propriétaire'] = author_link.get_text().strip() if author_link else author_elem.get_text().strip()
                else:
                    data['propriétaire'] = ""
                
                all_data.append(data)
                
            except Exception as e:
                print(f"⚠️ Erreur article: {e}")
                continue
        
        if page < max_pages:
            time.sleep(1)
    
    print(f"\n✅ Total motos scrapées: {len(all_data)}")
    return all_data

In [ ]:
def scrape_locations_brut(base_url: str, max_pages: int = None) -> List[Dict]:
    """
    Scrape les données brutes des locations (SANS NETTOYAGE) - TOUTES LES PAGES
    Variables: marque, année, prix, adresse, propriétaire
    """
    if max_pages is None:
        print("🔍 Détection du nombre total de pages...")
        max_pages = get_total_pages(base_url)
        print(f"✓ {max_pages} pages détectées\n")
    
    all_data = []
    
    for page in range(1, max_pages + 1):
        print(f"📄 Scraping page {page}/{max_pages}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url
        soup = get_page_content(url)
        
        if not soup:
            print(f"❌ Impossible de récupérer la page {page}, arrêt.")
            break
            
        articles = soup.find_all('div', class_='listings-cards__list-item')
        
        if not articles:
            print(f"⚠️ Aucun article trouvé sur la page {page}, arrêt.")
            break
            
        for article in articles:
            try:
                data = {}
                
                # V1: Marque - BRUT
                title_elem = article.find('h2', class_='listing-card__header__title')
                if title_elem:
                    title_link = title_elem.find('a')
                    data['marque'] = title_link.get_text().strip() if title_link else title_elem.get_text().strip()
                else:
                    data['marque'] = ""
                
                # V2: Année
                data['année'] = ""
                if data['marque']:
                    year_match = re.search(r'\b(19|20)\d{2}\b', data['marque'])
                    if year_match:
                        data['année'] = year_match.group()
                
                # V3: Prix - BRUT
                price_elem = article.find('h3', class_='listing-card__header__price')
                data['prix'] = price_elem.get_text().strip() if price_elem else ""
                
                # V4: Adresse - BRUT
                address_parts = []
                town_elem = article.find('span', class_='town-suburb')
                if town_elem:
                    address_parts.append(town_elem.get_text().strip())
                province_elem = article.find('span', class_='province')
                if province_elem:
                    address_parts.append(province_elem.get_text().strip())
                data['adresse'] = ' '.join(address_parts)
                
                # V5: Propriétaire - BRUT
                author_elem = article.find('p', class_='time-author')
                if author_elem:
                    author_link = author_elem.find('a')
                    data['propriétaire'] = author_link.get_text().strip() if author_link else author_elem.get_text().strip()
                else:
                    data['propriétaire'] = ""
                
                all_data.append(data)
                
            except Exception as e:
                print(f"⚠️ Erreur article: {e}")
                continue
        
        if page < max_pages:
            time.sleep(1)
    
    print(f"\n✅ Total locations scrapées: {len(all_data)}")
    return all_data

## Exécution du scraping SANS nettoyage - TOUTES LES PAGES

In [ ]:
# URLs
URL_VOITURES = "https://www.dakar-auto.com/voitures-4"
URL_MOTOS = "https://www.dakar-auto.com/motos-and-scooters-3"
URL_LOCATIONS = "https://www.dakar-auto.com/location-de-voitures-19"

print("=" * 80)
print("🚗 SCRAPING DES VOITURES (TOUTES LES PAGES - SANS NETTOYAGE)")
print("=" * 80)
voitures_brutes = scrape_voitures_brut(URL_VOITURES)
df_voitures_brut = pd.DataFrame(voitures_brutes)

print("\n" + "=" * 80)
print("🏍️ SCRAPING DES MOTOS (TOUTES LES PAGES - SANS NETTOYAGE)")
print("=" * 80)
motos_brutes = scrape_motos_brut(URL_MOTOS)
df_motos_brut = pd.DataFrame(motos_brutes)

print("\n" + "=" * 80)
print("🚙 SCRAPING DES LOCATIONS (TOUTES LES PAGES - SANS NETTOYAGE)")
print("=" * 80)
locations_brutes = scrape_locations_brut(URL_LOCATIONS)
df_locations_brut = pd.DataFrame(locations_brutes)

print("\n" + "=" * 80)
print("✅ SCRAPING TERMINÉ!")
print("=" * 80)
print(f"Voitures: {len(df_voitures_brut)} lignes")
print(f"Motos: {len(df_motos_brut)} lignes")
print(f"Locations: {len(df_locations_brut)} lignes")

🚗 SCRAPING DES VOITURES (TOUTES LES PAGES - SANS NETTOYAGE)
🔍 Détection du nombre total de pages...
Erreur lors de la récupération de https://www.dakar-auto.com/voitures-4: 404 Client Error: Not Found for url: https://dakar-auto.com/voitures-4
✓ 1 pages détectées

📄 Scraping page 1/1...
Erreur lors de la récupération de https://www.dakar-auto.com/voitures-4: 404 Client Error: Not Found for url: https://dakar-auto.com/voitures-4
❌ Impossible de récupérer la page 1, arrêt.

✅ Total voitures scrapées: 0

🏍️ SCRAPING DES MOTOS (TOUTES LES PAGES - SANS NETTOYAGE)
🔍 Détection du nombre total de pages...
Erreur lors de la récupération de https://www.dakar-auto.com/motos-and-scooters-3: 404 Client Error: Not Found for url: https://dakar-auto.com/motos-and-scooters-3
✓ 1 pages détectées

📄 Scraping page 1/1...
Erreur lors de la récupération de https://www.dakar-auto.com/motos-and-scooters-3: 404 Client Error: Not Found for url: https://dakar-auto.com/motos-and-scooters-3
❌ Impossible de récupér

## Aperçu des données brutes

In [ ]:
print("🚗 VOITURES (données brutes):")
print(df_voitures_brut.head())
print(f"\nShape: {df_voitures_brut.shape}")
print(f"Valeurs manquantes:\n{df_voitures_brut.isnull().sum()}")

🚗 VOITURES (données brutes):
Empty DataFrame
Columns: []
Index: []

Shape: (0, 0)
Valeurs manquantes:
Series([], dtype: float64)


In [ ]:
print("🏍️ MOTOS (données brutes):")
print(df_motos_brut.head())
print(f"\nShape: {df_motos_brut.shape}")
print(f"Valeurs manquantes:\n{df_motos_brut.isnull().sum()}")

🏍️ MOTOS (données brutes):
Empty DataFrame
Columns: []
Index: []

Shape: (0, 0)
Valeurs manquantes:
Series([], dtype: float64)


In [ ]:
print("🚙 LOCATIONS (données brutes):")
print(df_locations_brut.head())
print(f"\nShape: {df_locations_brut.shape}")
print(f"Valeurs manquantes:\n{df_locations_brut.isnull().sum()}")

🚙 LOCATIONS (données brutes):
Empty DataFrame
Columns: []
Index: []

Shape: (0, 0)
Valeurs manquantes:
Series([], dtype: float64)
